In [4]:
import credentials as cd
from fyers_apiv3 import fyersModel
import pandas as pd
import datetime as dt

In [49]:
with open("access.txt", "r") as a:
    access_token = a.read()
client_id = cd.client_id

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path=""
)

In [50]:
data = {
    "symbol": "NSE:NIFTY50-INDEX",
    "resolution": "1",
    "date_format": "1",
    "range_from": "2024-03-01",
    "range_to": "2024-05-09",
    "cont_flag": "1",
}

response = fyers.history(data=data)
print(response)
data = response["candles"]
df = pd.DataFrame(data)
print(df)

df.columns = ["date", "open", "high", "low", "close", "volume"]
df["date"] = pd.to_datetime(df["date"], unit="s")

df.date = df.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
print(df)
df["date"] = df["date"].dt.tz_localize(None)
df = df.set_index("date")

print(df)
df.to_csv("data.csv")
print(dt.datetime.now())

{'candles': [[1709264700, 22061.6, 22125.65, 22047.75, 22117.25, 0], [1709264760, 22117.55, 22124.05, 22117.5, 22118.4, 0], [1709264820, 22119.45, 22122.4, 22114.9, 22114.9, 0], [1709264880, 22113.65, 22122.65, 22112.35, 22121.75, 0], [1709264940, 22122.2, 22123.4, 22114.1, 22114.95, 0], [1709265000, 22115.1, 22119.2, 22105.75, 22115.15, 0], [1709265060, 22115.2, 22119.25, 22111, 22118.1, 0], [1709265120, 22118.85, 22122.35, 22113, 22120.35, 0], [1709265180, 22122.1, 22122.75, 22117.55, 22118.25, 0], [1709265240, 22119.15, 22124.4, 22117.25, 22124.4, 0], [1709265300, 22124.95, 22132, 22120.3, 22130.65, 0], [1709265360, 22128.9, 22129.65, 22124.5, 22125.45, 0], [1709265420, 22124.3, 22131.35, 22124.3, 22129.6, 0], [1709265480, 22128.35, 22132.9, 22126.75, 22127.85, 0], [1709265540, 22128.25, 22128.25, 22116.85, 22122.5, 0], [1709265600, 22122.1, 22129.05, 22119.95, 22123.65, 0], [1709265660, 22124.1, 22124.2, 22114.65, 22116.9, 0], [1709265720, 22117.35, 22122.45, 22117.35, 22120.2, 0],

In [19]:
def fetchOHLC(ticker, interval, duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = ticker
    data = {
        "symbol": instrument,
        "resolution": interval,
        "date_format": "1",
        "range_from": "2024-03-01",
        "range_to": "2024-05-09",
        "cont_flag": "1",
    }
    sdata = fyers.history(data)
    sdata = pd.DataFrame(sdata["candles"])
    sdata.columns = ["date", "open", "high", "low", "close", "volume"]
    sdata["date"] = pd.to_datetime(sdata["date"], unit="s")
    sdata.date = sdata.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
    sdata["date"] = sdata["date"].dt.tz_localize(None)
    sdata = sdata.set_index("date")
    return sdata


ticker = "NSE:NIFTY50-INDEX"
data = fetchOHLC(ticker, "1", 60)
print(data)

                         open      high       low     close  volume
date                                                               
2024-03-01 09:15:00  22048.30  22125.65  22047.75  22117.25       0
2024-03-01 09:16:00  22117.55  22124.05  22117.50  22118.40       0
2024-03-01 09:17:00  22119.45  22122.40  22114.90  22114.90       0
2024-03-01 09:18:00  22113.65  22122.65  22112.35  22121.75       0
2024-03-01 09:19:00  22122.20  22123.40  22114.10  22114.95       0
...                       ...       ...       ...       ...     ...
2024-05-09 15:25:00  21942.00  21948.60  21941.50  21948.50       0
2024-05-09 15:26:00  21948.70  21955.80  21947.85  21953.95       0
2024-05-09 15:27:00  21954.05  21957.50  21951.55  21957.35       0
2024-05-09 15:28:00  21956.35  21962.05  21951.30  21962.05       0
2024-05-09 15:29:00  21963.80  21968.15  21957.70  21968.15       0

[16980 rows x 5 columns]


In [52]:
def gethistory(symbol1, type, range_from, range_to):
    symbol = "NSE:" + symbol1 + "-" + type
    start = pd.to_datetime(range_from)
    end = pd.to_datetime(range_to)
    sdata = pd.DataFrame()
    while start <= end:
        end2 = start + pd.DateOffset(days=60)
        data = {
            "symbol": symbol,
            "resolution": "1",
            "date_format": "1",
            "range_from": start.strftime("%Y-%m-%d"),
            "range_to": end2.strftime("%Y-%m-%d"),
            "cont_flag": "1",
        }
        s = fyers.history(data)
        s = pd.DataFrame(s["candles"])
        sdata = pd.concat([sdata, s], ignore_index=True)
        start = end2 + dt.timedelta(1)
    sdata.columns = ["date", "open", "high", "low", "close", "volume"]
    sdata["date"] = pd.to_datetime(sdata["date"], unit="s")
    sdata.date = sdata.date.dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
    sdata["date"] = sdata["date"].dt.tz_localize(None)
    sdata = sdata.set_index("date")
    return sdata


data = gethistory("NIFTY50", "INDEX", "2024-03-01", "2024-05-09")
print(data)
data.to_csv("NIFTY50.csv")

                         open      high       low     close  volume
date                                                               
2024-03-01 09:15:00  22061.60  22125.65  22047.75  22117.25       0
2024-03-01 09:16:00  22117.55  22124.05  22117.50  22118.40       0
2024-03-01 09:17:00  22119.45  22122.40  22114.90  22114.90       0
2024-03-01 09:18:00  22113.65  22122.65  22112.35  22121.75       0
2024-03-01 09:19:00  22122.20  22123.40  22114.10  22114.95       0
...                       ...       ...       ...       ...     ...
2024-05-09 15:25:00  21942.00  21948.60  21941.50  21948.50       0
2024-05-09 15:26:00  21948.70  21955.80  21947.85  21953.95       0
2024-05-09 15:27:00  21954.05  21957.50  21951.55  21957.35       0
2024-05-09 15:28:00  21956.35  21962.05  21951.30  21962.05       0
2024-05-09 15:29:00  21963.80  21968.15  21957.70  21968.15       0

[16980 rows x 5 columns]


In [53]:
# # Define the EMA periods
# short_ema_period = 9
# long_ema_period = 13
# signal_line_period = 9

# # Calculate the short-period EMA
# data[f"EMA{short_ema_period}"] = (
#     data["close"].ewm(span=short_ema_period, adjust=False).mean()
# )

# # Calculate the long-period EMA
# data[f"EMA{long_ema_period}"] = (
#     data["close"].ewm(span=long_ema_period, adjust=False).mean()
# )

# # Calculate MACD (the difference between short-period EMA and long-period EMA)
# data["MACD"] = data[f"EMA{short_ema_period}"] - data[f"EMA{long_ema_period}"]

# # Calculate the signal line (the EMA of MACD)
# data["Signal_Line"] = data["MACD"].ewm(span=signal_line_period, adjust=False).mean()

In [ ]:
print(data)

In [55]:
# # Initialize an empty list to store the signals
# signals = []

# # Check for MACD and Signal Line crossovers in the last two rows
# last_row = data.iloc[-1]
# second_last_row = data.iloc[-2]

# if (
#     second_last_row["MACD"] > second_last_row["Signal_Line"]
#     and last_row["MACD"] < last_row["Signal_Line"]
# ):
#     print("Cross Below Signal Line")
#     signals.append(
#         {
#             "timestamp": last_row.name,
#             "price": last_row["close"],
#             "signal": "sell",
#             "MACD": last_row["MACD"],
#             "Signal_Line": last_row["Signal_Line"],
#         }
#     )
# elif (
#     second_last_row["MACD"] < second_last_row["Signal_Line"]
#     and last_row["MACD"] > last_row["Signal_Line"]
# ):
#     print("Cross Above Signal Line")
#     signals.append(
#         {
#             "timestamp": last_row.name,
#             "price": last_row["close"],
#             "signal": "buy",
#             "MACD": last_row["MACD"],
#             "Signal_Line": last_row["Signal_Line"],
#         }
#     )
# else:
#     print("No Crossover")

# # Convert the list of signals to a DataFrame for easier manipulation
# signals_df = pd.DataFrame(signals)
# print(signals_df)

In [71]:
import ta


# Calculate the 9-period and 13-period EMA of the closing prices
df["EMA9"] = ta.trend.ema_indicator(df.close, window=9)
df["EMA13"] = ta.trend.ema_indicator(df.close, window=13)

# Calculate the MACD with fast period of 9, slow period of 13, and signal period of 9
macd = ta.trend.MACD(df.close, window_fast=9, window_slow=13, window_sign=9)
df["MACD"], df["MACD_signal"], df["MACD_hist"] = (
    macd.macd(),
    macd.macd_signal(),
    macd.macd_diff(),
)

# Initialize an empty list to store the signals
signals = []

# Loop over all rows in the data (starting from the second one)
for i in range(
    1, len(df) - 1
):  # Adjusted to len(df)-1 to avoid index error in the next line
    # Get the current row, the previous row, and the next row
    current_row = df.iloc[i]
    previous_row = df.iloc[i - 1]
    next_row = df.iloc[i + 1]  # Added next_row

    # Identify the trend based on the MACD and Signal Line
    if current_row["MACD"] > current_row["MACD_signal"]:
        trend = "up"
    else:
        trend = "down"

    # Check for EMA crossovers
    if (
        previous_row["EMA9"] > previous_row["EMA13"]
        and current_row["EMA9"] < current_row["EMA13"]
        and trend == "down"
    ):
        signals.append(
            {
                "timestamp": current_row.name,
                "price": current_row["close"],
                "signal": "sell",
                "MACD": current_row["MACD"],
                "Signal_Line": current_row["MACD_signal"],
                "trend": trend,
            }
        )
    elif (
        previous_row["EMA9"] < previous_row["EMA13"]
        and current_row["EMA9"] > current_row["EMA13"]
        and trend == "up"
    ):
        signals.append(
            {
                "timestamp": current_row.name,
                "price": current_row["close"],
                "signal": "buy",
                "MACD": current_row["MACD"],
                "Signal_Line": current_row["MACD_signal"],
                "trend": trend,
            }
        )

# Convert the list of signals to a DataFrame for easier manipulation
signals_df = pd.DataFrame(signals)

# Calculate profit/loss
signals_df["profit_loss"] = signals_df["price"].diff()
# For 'buy' signals, we need to take the negative of the profit/loss
signals_df.loc[signals_df["signal"] == "buy", "profit_loss"] *= -1

# Output the DataFrame to a CSV file
signals_df.to_csv("signals.csv", index=False)

print(signals_df)

              timestamp     price signal      MACD  Signal_Line trend  \
0   2024-03-01 10:37:00  22193.50   sell -0.038542     0.890677  down   
1   2024-03-01 10:53:00  22192.40    buy  0.158421    -0.687299    up   
2   2024-03-01 10:59:00  22172.45   sell -0.671781     0.139517  down   
3   2024-03-01 11:09:00  22191.20    buy  0.147055    -0.358836    up   
4   2024-03-01 12:03:00  22275.85   sell -0.211512     1.422534  down   
..                  ...       ...    ...       ...          ...   ...   
891 2024-05-09 14:08:00  22060.30    buy  0.794929    -0.773288    up   
892 2024-05-09 14:19:00  22047.95   sell -0.054060     0.686606  down   
893 2024-05-09 14:28:00  22042.85   sell -0.730840     0.348097  down   
894 2024-05-09 14:54:00  22008.65    buy  0.048029    -2.491871    up   
895 2024-05-09 15:28:00  21962.05    buy  0.068718    -2.112942    up   

     profit_loss  
0            NaN  
1           1.10  
2         -19.95  
3         -18.75  
4          84.65  
..       